In [1]:
import os
import re
import pickle
import time

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix

%matplotlib inline

# Custom modules
import const
import func

## Load data

In [6]:
print const.TRAIN_FILES
print const.LOOK_UP_TABLE

['train_numeric', 'train_categorical_to_num', 'train_date']
./data/date_feat_lut_V2.csv


In [82]:
lut = pd.read_csv(const.LOOK_UP_TABLE)
lut['station_V2'] = lut['station_V2'].round(3)
lut.head()

,line,station,feature_nr,feat_nr_dat,name_dat,name_cat,name_num,station_V2
0,0,0,0,1.0,L0_S0_D1,NaN,L0_S0_F0,0.0
1,0,0,2,3.0,L0_S0_D3,NaN,L0_S0_F2,0.0
2,0,0,4,5.0,L0_S0_D5,NaN,L0_S0_F4,0.0
3,0,0,6,7.0,L0_S0_D7,NaN,L0_S0_F6,0.0
4,0,0,8,9.0,L0_S0_D9,NaN,L0_S0_F8,0.0


In [ ]:
def post_process(df, mergecol_lut):
    
    df.set_index('Id', inplace=True)
    df = df.transpose()
    df = df.merge(lut[['station_V2', 'feature_nr', mergecol_lut]], left_index=True, right_on=mergecol_lut, how='right')
    df.set_index('feature_nr', inplace=True)
    df.drop(mergecol_lut, axis=1, inplace=True)
    return df

[num_dat, cat_dat, date_dat] = [func.load_data_file(f, ftype='csv', nrows=250000) for f in const.TRAIN_FILES]


num_dat = post_process(num_dat, 'name_num')
cat_dat = post_process(cat_dat, 'name_cat')
date_dat = post_process(date_dat, 'name_dat')

Returning train_numeric.csv
Returning train_categorical_to_num.csv
Returning train_date.csv


In [ ]:
print('Numeric data size: {}'.format(num_dat.shape))
print('Categorical data size: {}'.format(cat_dat.shape))
print('Date data size: {}'.format(date_dat.shape))

In [ ]:
num_dat_cnt = num_dat.groupby('station_V2').count()
cat_dat_cnt = cat_dat.groupby('station_V2').count()
date_dat_cnt = date_dat.groupby('station_V2').count()

In [ ]:
print('Numeric data size: {}'.format(num_dat_cnt.shape))
print('Categorical data size: {}'.format(cat_dat_cnt.shape))
print('Date data size: {}'.format(date_dat_cnt.shape))

In [ ]:
# So are there any station where
((date_dat_cnt==0) & ((num_dat_cnt>0) | (cat_dat_cnt>0))).sum().sum()

In [ ]:
26743. / (100000 * 129)

In [ ]:
_*100